In [3]:
import requests
import json
import pandas as pd
from datetime import datetime

# Point to correct subgraph URL
url = 'https://gateway.thegraph.com/api/[api-key]/subgraphs/id/0x9bde7bf4d5b13ef94373ced7c8ee0be59735a298-2'

# initialize full_data
df1_fulldata = pd.DataFrame()
df2_fulldata = pd.DataFrame()
df_data = pd.DataFrame()

for i in range(0,40) :
    # Set query (which uses text input to specify ETH wallet addresses)
    query = '''query {
      swaps(orderBy: timestamp orderDirection: desc, first:1000, skip:%s){
        id
        timestamp
        pool{
          token0{
            symbol
          }
          token1{
            symbol
          }
          txCount
        }
        amount0
        amount1
        amountUSD
        token0{
            id
            totalSupply
            volume
            volumeUSD
            totalValueLocked
            totalValueLockedUSD
        }
        token1{
            id
            totalSupply
            volume
            volumeUSD
            totalValueLocked
            totalValueLockedUSD
        }
      }
    }
    ''' % (i*1000)


    # Make the request
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    
    # JSON adjustment
    json_data = json.loads(r.text)
    
    # extract JSON to convert to a dataframe
    df_data = json_data['data']['swaps']
    df1_fulldata = pd.DataFrame(df_data)
        
    df2_fulldata = pd.concat([df2_fulldata,df1_fulldata])
    
print(df2_fulldata)  
#df2_fulldata.to_excel(r'C:\Users\Nehal.Jain\Documents\Nehal_Personal\Project\Blockchain\Dataset2.xlsx', sheet_name='Data', index = False)






                            amount0                  amount1  \
0                               100    -1.531729427617388149   
1     -613631056.738320065993892439     1.531729427617388149   
2                      -54630.40469                       22   
3           -997.561222142511463809     6.492391575948414119   
4                    -247980.645808                      100   
5                      23925.247082    -9.624580964185377082   
6                     448077.249269  -180.432817122745652731   
7          -1242.225611096794089595         8.04512125431614   
8             795.55655794180221516    -0.364852273348374387   
9          -1672.809010471901197708    10.861534449457663492   
10           -16.163808094629123451                   193550   
11          1023.528736028138613397    -1.091319310114951834   
12             0.104304741723011173   -42.623133194386102466   
13                             0.22          -27579.79005668   
14                     -4498.060132     

In [4]:

#Removing NULLS - Data Cleaning
df = df2_fulldata.dropna()
#print(df)

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Splitting 'pool' column into 'From_Currency' and 'To_Currency'

df[['From_Currency','To_Currency','txCount']] = df.pool.apply(
   lambda x: pd.Series(str(x).split(",")))

#Splitting 'Token' columns
df[['From_Token_id','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD']] = df.token0.apply(
   lambda z: pd.Series(str(z).split(",")))
df[['To_Token_id','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD']] = df.token1.apply(
   lambda n: pd.Series(str(n).split(",")))

#Removing extra data from columns From_Currency and To_Currency : Cleaning
df['From_Currency'] = df['From_Currency'].map(lambda x: str(x)[23:-2])
df['To_Currency'] = df['To_Currency'].map(lambda y: str(y)[23:-2])
df['txCount'] = df['txCount'].map(lambda y: str(y)[13:-2])
df['From_Token_id'] = df['From_Token_id'].map(lambda y: str(y)[8:-1])
df['To_Token_id'] = df['To_Token_id'].map(lambda y: str(y)[8:-1])
df['From_Token_totalSupply'] = df['From_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['To_Token_totalSupply'] = df['To_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['From_Token_totalValueLocked'] = df['From_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['From_Token_totalValueLockedUSD'] = df['From_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['To_Token_totalValueLocked'] = df['To_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['To_Token_totalValueLockedUSD'] = df['To_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['From_Token_volume'] = df['From_Token_volume'].map(lambda y: str(y)[12:-1])
df['From_Token_volumeUSD'] = df['From_Token_volumeUSD'].map(lambda y: str(y)[15:-2])
df['To_Token_volume'] = df['To_Token_volume'].map(lambda y: str(y)[12:-1])
df['To_Token_volumeUSD'] = df['To_Token_volumeUSD'].map(lambda y: str(y)[15:-2])

#print(df)

#Dropping the pool, token columns
df = df.drop(['pool'], axis=1)
df = df.drop(['token0'], axis=1)
df = df.drop(['token1'], axis=1)

#Cleaning and re-ordering of data
df = df.rename(columns={'amount0': 'From_Currency_Amount', 'amount1': 'To_Currency_Amount', 'amountUSD': 'Amount_USD', 'id': 'TransactionID','timestamp': 'Date_Time'})
df = df [['TransactionID','From_Token_id','From_Currency_Amount','From_Currency','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD','To_Token_id','To_Currency_Amount','To_Currency','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD','Amount_USD','txCount','Date_Time']]
#df.head()
print(df)


                                         TransactionID  \
0    0xfd066a91d1179334d3c873f4754db43f07a32c5854a4...   
1    0xfd066a91d1179334d3c873f4754db43f07a32c5854a4...   
2    0xa9f9ad40813fb1e9d95f8ad27ca114f44a945e00ca91...   
3    0x8b77ac83747d710f9f8bbec7b43666d7738baf7a81e7...   
4    0x7b85c225601764c3bab34901c284cc9b1655b388c75a...   
5    0x6505924d6c1492c14c2a3fdab505fb02b21ccb82dace...   
6    0x5c92fd4217dd96368af9e0f0a8b1cd8f1d86f83e46be...   
7    0xe8011e6f7be57857885706b9dd81834bd54afedc4086...   
8    0xdd1a5ea915cf67c0d377f81db636b421f6d78e0a5040...   
9    0xd63d7890d0d93b0fbd980c43a98bddaf329eecc9010d...   
10   0xa02f124afb3734bd47a9f611b67cacc919535824c817...   
11   0x7f18f828b2b4839e19692393881efa14938fad3fe4ab...   
12   0x5d4939fe887166e0571ff36b085fe4bdb4ae99d480a9...   
13   0x4be2b7d5516c264e24031beebc30251922549a050f4a...   
14   0xea1aa029061b6421576129eeac0a50002af5c3345f4c...   
15   0xe876dfabd737763bf465773a63d8f810c57ac931e196...   
16   0xe876dfa

In [5]:
df.head()

,TransactionID,From_Token_id,From_Currency_Amount,From_Currency,From_Token_totalSupply,From_Token_totalValueLocked,From_Token_totalValueLockedUSD,From_Token_volume,From_Token_volumeUSD,To_Token_id,To_Currency_Amount,To_Currency,To_Token_totalSupply,To_Token_totalValueLocked,To_Token_totalValueLockedUSD,To_Token_volume,To_Token_volumeUSD,Amount_USD,txCount,Date_Time
0,0xfd066a91d1179334d3c873f4754db43f07a32c5854a4...,0xbb0e17ef65f82ab018d8edd776e8dd940327b28b,100,AXS,28240,169767.790356315931597255,6459383.98879963081812835913888316,23147486.498527749506108821,517100340.11285199620664945244578,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-1.531729427617388149,WETH,19848,283921.735924650694681457,703224459.3935353220835271498668676,30396597.448773139091259371,72752656319.98392398848412754754501,3793.825770913394281491967347358172,77607,2021-08-03 20:33:25
1,0xfd066a91d1179334d3c873f4754db43f07a32c5854a4...,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,-613631056.738320065993892439,SHIB,18240,769831516428.39199711340926074,4704697.274907788614463861557880978,141933277687040.595082363275766932,1532635870.246833313930075524318631,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.531729427617388149,WETH,19848,283921.735924650694681457,703224459.3935353220835271498668676,30396597.448773139091259371,72752656319.98392398848412754754501,3793.825770913394281491967347358172,160977,2021-08-03 20:33:25
2,0xa9f9ad40813fb1e9d95f8ad27ca114f44a945e00ca91...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,-54630.40469,USDC,19312,444213269.899155,444213269.8991550000000000000000002,50238542410.069755,50239694286.23897844652637055219501,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,22,WETH,19848,283921.735924650694681457,703224459.3935353220835271498668676,30396597.448773139091259371,72752656319.98392398848412754754501,54560.27757017327111048324644203855,430077,2021-08-03 20:33:25
3,0x8b77ac83747d710f9f8bbec7b43666d7738baf7a81e7...,0xbc396689893d065f41bc2c6ecbee5e0085233447,-997.561222142511463809,PERP,28240,330739.6299237254665293,5320143.006798407807775059833172331,18081039.899302704395427245,154078279.9228819464818301543612713,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,6.492391575948414119,WETH,19848,283921.735924650694681457,703224459.3935353220835271498668676,30396597.448773139091259371,72752656319.98392398848412754754501,16080.51789799961702633306890195789,10846,2021-08-03 20:33:25
4,0x7b85c225601764c3bab34901c284cc9b1655b388c75a...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,-247980.645808,USDC,19312,444213269.899155,444213269.8991550000000000000000002,50238542410.069755,50239694286.23897844652637055219501,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100,WETH,19848,283921.735924650694681457,703224459.3935353220835271498668676,30396597.448773139091259371,72752656319.98392398848412754754501,247831.5739275148686840147565547206,430077,2021-08-03 20:33:25
